# MBTA Analysis

## Initialization

Imports:

In [2]:
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

Read data files:

In [3]:
def get_mbta(name):
    return pd.read_csv(f'../MBTA_GTFS/{name}.txt')

trips = get_mbta('trips')
stops = get_mbta('stops')

/var/folders/32/ydwhcwss24b_556zw0h36nrr0000gn/T/ipykernel_13602/2331019837.py:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(f'./MBTA_GTFS/{name}.txt')


Check data:

In [4]:
stops[:30]
# stops[:30]

,stop_id,stop_code,stop_name,stop_desc,platform_code,platform_name,stop_lat,stop_lon,zone_id,stop_address,stop_url,level_id,location_type,parent_station,wheelchair_boarding,municipality,on_street,at_street,vehicle_type
0,1,1.0,Washington St opp Ruggles St,NaN,NaN,NaN,42.330957,-71.082754,ExpressBus-Downtown,NaN,https://www.mbta.com/stops/1,NaN,0,NaN,1,Boston,Washington Street,Ruggles Street,3.0
1,10,10.0,Theo Glynn Way @ Newmarket Sq,NaN,NaN,NaN,42.330555,-71.068787,LocalBus,NaN,https://www.mbta.com/stops/10,NaN,0,NaN,1,Boston,Theodore Glynn Way,Newmarket Square,3.0
2,10000,10000.0,Tremont St opp Temple Pl,NaN,NaN,NaN,42.355692,-71.062911,LocalBus,NaN,https://www.mbta.com/stops/10000,NaN,0,NaN,1,Boston,Tremont Street,Temple Place,3.0
3,10003,10003.0,Albany St opp Randall St,NaN,NaN,NaN,42.331591,-71.076237,LocalBus,NaN,https://www.mbta.com/stops/10003,NaN,0,NaN,1,Boston,Albany Street,Randall Street,3.0
4,10005,10005.0,Albany St opp E Concord St,NaN,NaN,NaN,42.335017,-71.071280,LocalBus,NaN,https://www.mbta.com/stops/10005,NaN,0,NaN,1,Boston,Albany Street,NaN,3.0
5,10006,10006.0,Albany St opp E Newton St,NaN,NaN,NaN,42.335814,-71.070135,LocalBus,NaN,https://www.mbta.com/stops/10006,NaN,0,NaN,1,Boston,Albany Street,East Newton Street,3.0
6,10007,10007.0,540 Albany St opp Wareham St,NaN,NaN,NaN,42.338372,-71.066556,LocalBus,NaN,https://www.mbta.com/stops/10007,NaN,0,NaN,1,Boston,Albany Street,Wareham Street,3.0
7,10008,10008.0,Albany St opp Randolph St,NaN,NaN,NaN,42.339915,-71.064356,LocalBus,NaN,https://www.mbta.com/stops/10008,NaN,0,NaN,1,Boston,Albany Street,NaN,3.0
8,10009,10009.0,High St @ Duck Lane,NaN,NaN,NaN,42.214986,-70.915924,LocalBus,NaN,https://www.mbta.com/stops/10009,NaN,0,NaN,1,Hingham,High Street,NaN,3.0
9,10010,10010.0,Southampton St @ Newmarket St,NaN,NaN,NaN,42.330944,-71.064621,LocalBus,NaN,https://www.mbta.com/stops/10010,NaN,0,NaN,1,Boston,Southampton Street,Newmarket Square,3.0


Clean data:

In [64]:
silver_stops = stops[stops['stop_desc'].str.contains('Silver Line')==True] # bus
blue_stops = stops[stops['stop_desc'].str.contains('Blue Line')==True]
red_stops = stops[stops['stop_desc'].str.contains('Red Line')==True]
green_stops = stops[stops['stop_desc'].str.contains('Green Line')==True] # light rail
orange_stops = stops[stops['stop_desc'].str.contains('Orange Line')==True]

included_stops = [
    silver_stops,
    blue_stops,
    red_stops,
    green_stops,
    orange_stops,
]

rail_stops = pd.concat(included_stops)

stops_no_silver = stops[stops['zone_id'] == 'RapidTransit']
stops_no_silver[stops['platform_code'].isna()]
# stops_no_silver[stops['stop_name'] == 'Alewife']

/var/folders/32/ydwhcwss24b_556zw0h36nrr0000gn/T/ipykernel_7640/3156141115.py:18: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  stops_no_silver[stops['platform_code'].isna()]


,stop_id,stop_code,stop_name,stop_desc,platform_code,platform_name,stop_lat,stop_lon,zone_id,stop_address,stop_url,level_id,location_type,parent_station,wheelchair_boarding,municipality,on_street,at_street,vehicle_type
7236,70061,70061.0,Alewife,Alewife - Red Line,NaN,Red Line,42.396158,-71.139971,RapidTransit,NaN,https://www.mbta.com/stops/place-alfcl,level_-2_platform,0,place-alfcl,1,Cambridge,NaN,NaN,1.0
7247,70126,70126.0,Allston Street,Allston Street - Green Line - Park Street & North,NaN,Park Street & North,42.348649,-71.137881,RapidTransit,NaN,https://www.mbta.com/stops/place-alsgr,level_median,0,place-alsgr,2,Boston,NaN,NaN,0.0
7248,70127,70127.0,Allston Street,Allston Street - Green Line - (B) Boston College,NaN,Boston College,42.348546,-71.137362,RapidTransit,NaN,https://www.mbta.com/stops/place-alsgr,level_median,0,place-alsgr,2,Boston,NaN,NaN,0.0
7250,170140,170140.0,Amory Street,Amory Street - Green Line - Park Street & North,NaN,Park Street & North,42.350901,-71.114318,RapidTransit,NaN,https://www.mbta.com/stops/place-amory,level_median,0,place-amory,1,Boston,NaN,NaN,0.0
7251,170141,170141.0,Amory Street,Amory Street - Green Line - (B) Boston College,NaN,Boston College,42.351066,-71.115027,RapidTransit,NaN,https://www.mbta.com/stops/place-amory,level_median,0,place-amory,1,Boston,NaN,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9578,70100,70100.0,Wollaston,Wollaston - Red Line - Alewife,NaN,Alewife,42.266788,-71.020457,RapidTransit,NaN,https://www.mbta.com/stops/place-wlsta,level_1_platform,0,place-wlsta,1,Quincy,NaN,NaN,1.0
9667,70162,70162.0,Woodland,Woodland - Green Line - Park Street & North,NaN,Park Street & North,42.332703,-71.243055,RapidTransit,NaN,https://www.mbta.com/stops/place-woodl,level_ground,0,place-woodl,1,Newton,NaN,NaN,0.0
9668,70163,70163.0,Woodland,Woodland - Green Line - (D) Riverside,NaN,Riverside,42.333094,-71.243659,RapidTransit,NaN,https://www.mbta.com/stops/place-woodl,level_ground,0,place-woodl,1,Newton,NaN,NaN,0.0
9672,70124,70124.0,Warren Street,Warren Street - Green Line - Park Street & North,NaN,Park Street & North,42.348285,-71.140436,RapidTransit,NaN,https://www.mbta.com/stops/place-wrnst,level_median,0,place-wrnst,2,Boston,NaN,NaN,0.0


## Creating the Graph

In [56]:
network = nx.Graph()

stops_set = set(stops_no_silver['stop_name'].tolist())
network.add_nodes_from(stops_set)

# print(len(stops_set))
# for i in sorted(stops_set):
#     print(i)

edge_list = []
for i in range(len(stops)-1):
    stop1 = int(stops['stop_id'].iloc[[i]])
    stop2 = int(stops['stop_id'].iloc[[i+1]])

    if stop1 + 1 == stop2:
        edge_list.append((stop1, stop2))



125
Airport
Alewife
Allston Street
Amory Street
Andrew
Aquarium
Arlington
Ashmont
Assembly
Babcock Street
Back Bay
Back of the Hill
Ball Square
Beachmont
Beaconsfield
Blandford Street
Boston College
Boston University Central
Boston University East
Bowdoin
Boylston
Braintree
Brandon Hall
Brigham Circle
Broadway
Brookline Hills
Brookline Village
Butler
Capen Street
Cedar Grove
Central
Central Avenue
Charles/MGH
Chestnut Hill
Chestnut Hill Avenue
Chinatown
Chiswick Road
Cleveland Circle
Community College
Coolidge Corner
Copley
Davis
Dean Road
Downtown Crossing
East Somerville
Eliot
Englewood Avenue
Fairbanks Street
Fenway
Fenwood Road
Fields Corner
Forest Hills
Gilman Square
Government Center
Green Street
Griggs Street
Harvard
Harvard Avenue
Hawes Street
Haymarket
Heath Street
Hynes Convention Center
JFK/UMass
Jackson Square
Kendall/MIT
Kenmore
Kent Street
Lechmere
Longwood
Longwood Medical Area
Magoun Square
Malden Center
Massachusetts Avenue
Mattapan
Maverick
Medford/Tufts
Milton
Missio